<a href="https://colab.research.google.com/github/gnaneswar-18/fs-10/blob/main/fs_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd;
from sklearn.preprocessing import LabelEncoder
data=pd.read_csv('/content/drive/MyDrive/Copy of melbourne_housing_raw.csv')
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
for column in data.keys():
  print(data[column].dtype)
  if data[column].dtype == 'object':
    data[column] = label_encoder.fit_transform(data[column])


In [ ]:
x=data.drop('Propertycount',axis=1)
y=data['Propertycount']
print(x.shape)
print(y.shape)
if y.isnull().any():
    y = y.dropna()
    x = x.loc[y.index]
print(x.shape)
print(y.shape)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
model = RandomForestRegressor(n_estimators=10, random_state=42)
def evaluate_model(X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

mae_list = []
num_features_list = []

current_features = list(X_train.columns)
initial_mae = evaluate_model(X_train[current_features], X_test[current_features], y_train, y_test)
mae_list.append(initial_mae)
num_features_list.append(len(current_features))

print(f"initial mae with all features: {initial_mae}")

In [ ]:
for _ in range(len(current_features) - 1):
    importances = model.feature_importances_
    least_important_idx = np.argmin(importances)
    least_important_feature = current_features[least_important_idx]
    current_features.pop(least_important_idx)
    new_mae = evaluate_model(X_train[current_features], X_test[current_features], y_train, y_test)


    mae_list.append(new_mae)
    num_features_list.append(len(current_features))
    print(f"removed {least_important_feature}, mae: {new_mae}, features left: {len(current_features)}")

In [ ]:
plt.plot(num_features_list, mae_list, marker='o')
plt.title('Impact of Feature Elimination on Model Accuracy')
plt.xlabel('Number of Features')
plt.ylabel('Mean Absolute Error (MAE)')
plt.gca().invert_xaxis()
plt.show()